In [137]:
#basic imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
device = torch.device("mps")
warnings.filterwarnings('ignore')
%matplotlib inline

In [129]:
df = pd.read_csv('StudentPerformanceFactors.csv')

In [3]:
df.head()

,Hours_Studied,Attendance,Parental_Involvement,Access_to_Resources,Extracurricular_Activities,Sleep_Hours,Previous_Scores,Motivation_Level,Internet_Access,Tutoring_Sessions,Family_Income,Teacher_Quality,School_Type,Peer_Influence,Physical_Activity,Learning_Disabilities,Parental_Education_Level,Distance_from_Home,Gender,Exam_Score
0,23,84,Low,High,No,7,73,Low,Yes,0,Low,Medium,Public,Positive,3,No,High School,Near,Male,67
1,19,64,Low,Medium,No,8,59,Low,Yes,2,Medium,Medium,Public,Negative,4,No,College,Moderate,Female,61
2,24,98,Medium,Medium,Yes,7,91,Medium,Yes,2,Medium,Medium,Public,Neutral,4,No,Postgraduate,Near,Male,74
3,29,89,Low,Medium,Yes,8,98,Medium,Yes,1,Medium,Medium,Public,Negative,4,No,High School,Moderate,Male,71
4,19,92,Medium,Medium,Yes,6,65,Medium,Yes,3,Medium,High,Public,Neutral,4,No,College,Near,Female,70


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6607 entries, 0 to 6606
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Hours_Studied               6607 non-null   int64 
 1   Attendance                  6607 non-null   int64 
 2   Parental_Involvement        6607 non-null   object
 3   Access_to_Resources         6607 non-null   object
 4   Extracurricular_Activities  6607 non-null   object
 5   Sleep_Hours                 6607 non-null   int64 
 6   Previous_Scores             6607 non-null   int64 
 7   Motivation_Level            6607 non-null   object
 8   Internet_Access             6607 non-null   object
 9   Tutoring_Sessions           6607 non-null   int64 
 10  Family_Income               6607 non-null   object
 11  Teacher_Quality             6529 non-null   object
 12  School_Type                 6607 non-null   object
 13  Peer_Influence              6607 non-null   obje

In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Hours_Studied,6607.0,19.975329,5.990594,1.0,16.0,20.0,24.0,44.0
Attendance,6607.0,79.977448,11.547475,60.0,70.0,80.0,90.0,100.0
Sleep_Hours,6607.0,7.029060,1.468120,4.0,6.0,7.0,8.0,10.0
Previous_Scores,6607.0,75.070531,14.399784,50.0,63.0,75.0,88.0,100.0
Tutoring_Sessions,6607.0,1.493719,1.230570,0.0,1.0,1.0,2.0,8.0
Physical_Activity,6607.0,2.967610,1.031231,0.0,2.0,3.0,4.0,6.0
Exam_Score,6607.0,67.235659,3.890456,55.0,65.0,67.0,69.0,101.0


In [6]:
#dropping duplicates
df.drop_duplicates(inplace=True)

In [7]:
#null values
df.isna().sum().sort_values(ascending=False).head(5)

Parental_Education_Level    90
Teacher_Quality             78
Distance_from_Home          67
Hours_Studied                0
Attendance                   0
dtype: int64

In [8]:
#imputing missing values
print(df['Parental_Education_Level'].value_counts(),"\n")
print(df['Teacher_Quality'].value_counts(),"\n")
print(df['Distance_from_Home'].value_counts(),"\n")

Parental_Education_Level
High School     3223
College         1989
Postgraduate    1305
Name: count, dtype: int64 

Teacher_Quality
Medium    3925
High      1947
Low        657
Name: count, dtype: int64 

Distance_from_Home
Near        3884
Moderate    1998
Far          658
Name: count, dtype: int64 



In [130]:
#filling with mode of each variable
df['Parental_Education_Level'].fillna(df['Parental_Education_Level'].mode()[0], inplace=True)
df['Teacher_Quality'].fillna(df['Teacher_Quality'].mode()[0], inplace=True)
df['Distance_from_Home'].fillna(df['Distance_from_Home'].mode()[0], inplace=True)

In [10]:
df.isna().sum().sort_values(ascending=False).head(5)

Hours_Studied               0
Attendance                  0
Gender                      0
Distance_from_Home          0
Parental_Education_Level    0
dtype: int64

### dividing features into categorical and numerical

In [11]:
cat_cols = ['Parental_Involvement','Access_to_Resources','Extracurricular_Activities','Motivation_Level','Internet_Access','Family_Income','Teacher_Quality','School_Type','Peer_Influence','Learning_Disabilities','Parental_Education_Level','Distance_from_Home','Gender']
num_cols = ['Hours_Studied', 'Attendance','Sleep_Hours','Previous_Scores','Tutoring_Sessions','Physical_Activity']
target_col = ['Exam_Score']

### Coverting categorical variables

In [12]:
df[cat_cols].head()

,Parental_Involvement,Access_to_Resources,Extracurricular_Activities,Motivation_Level,Internet_Access,Family_Income,Teacher_Quality,School_Type,Peer_Influence,Learning_Disabilities,Parental_Education_Level,Distance_from_Home,Gender
0,Low,High,No,Low,Yes,Low,Medium,Public,Positive,No,High School,Near,Male
1,Low,Medium,No,Low,Yes,Medium,Medium,Public,Negative,No,College,Moderate,Female
2,Medium,Medium,Yes,Medium,Yes,Medium,Medium,Public,Neutral,No,Postgraduate,Near,Male
3,Low,Medium,Yes,Medium,Yes,Medium,Medium,Public,Negative,No,High School,Moderate,Male
4,Medium,Medium,Yes,Medium,Yes,Medium,High,Public,Neutral,No,College,Near,Female


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6607 entries, 0 to 6606
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Hours_Studied               6607 non-null   int64 
 1   Attendance                  6607 non-null   int64 
 2   Parental_Involvement        6607 non-null   object
 3   Access_to_Resources         6607 non-null   object
 4   Extracurricular_Activities  6607 non-null   object
 5   Sleep_Hours                 6607 non-null   int64 
 6   Previous_Scores             6607 non-null   int64 
 7   Motivation_Level            6607 non-null   object
 8   Internet_Access             6607 non-null   object
 9   Tutoring_Sessions           6607 non-null   int64 
 10  Family_Income               6607 non-null   object
 11  Teacher_Quality             6607 non-null   object
 12  School_Type                 6607 non-null   object
 13  Peer_Influence              6607 non-null   obje

In [14]:
#max value for any variable is less than 256, thus we can convert to int8 without losing any info
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Hours_Studied,6607.0,19.975329,5.990594,1.0,16.0,20.0,24.0,44.0
Attendance,6607.0,79.977448,11.547475,60.0,70.0,80.0,90.0,100.0
Sleep_Hours,6607.0,7.029060,1.468120,4.0,6.0,7.0,8.0,10.0
Previous_Scores,6607.0,75.070531,14.399784,50.0,63.0,75.0,88.0,100.0
Tutoring_Sessions,6607.0,1.493719,1.230570,0.0,1.0,1.0,2.0,8.0
Physical_Activity,6607.0,2.967610,1.031231,0.0,2.0,3.0,4.0,6.0
Exam_Score,6607.0,67.235659,3.890456,55.0,65.0,67.0,69.0,101.0


In [15]:
for col in list(df.columns):
    if df[col].dtype == 'int64':
        df[col] = df[col].astype('int8')

In [16]:
df.head()

,Hours_Studied,Attendance,Parental_Involvement,Access_to_Resources,Extracurricular_Activities,Sleep_Hours,Previous_Scores,Motivation_Level,Internet_Access,Tutoring_Sessions,Family_Income,Teacher_Quality,School_Type,Peer_Influence,Physical_Activity,Learning_Disabilities,Parental_Education_Level,Distance_from_Home,Gender,Exam_Score
0,23,84,Low,High,No,7,73,Low,Yes,0,Low,Medium,Public,Positive,3,No,High School,Near,Male,67
1,19,64,Low,Medium,No,8,59,Low,Yes,2,Medium,Medium,Public,Negative,4,No,College,Moderate,Female,61
2,24,98,Medium,Medium,Yes,7,91,Medium,Yes,2,Medium,Medium,Public,Neutral,4,No,Postgraduate,Near,Male,74
3,29,89,Low,Medium,Yes,8,98,Medium,Yes,1,Medium,Medium,Public,Negative,4,No,High School,Moderate,Male,71
4,19,92,Medium,Medium,Yes,6,65,Medium,Yes,3,Medium,High,Public,Neutral,4,No,College,Near,Female,70


In [17]:
for col in cat_cols:
    df[col] = df[col].astype('category')

In [18]:
from sklearn.model_selection import train_test_split
train_df,test_df = train_test_split(df,test_size=0.2)

In [19]:
cat_train = np.stack([train_df[col].cat.codes for col in cat_cols],axis=1)
cat_train = torch.LongTensor(cat_train).to(device)
cat_test = np.stack([test_df[col].cat.codes for col in cat_cols],axis=1)
cat_test = torch.LongTensor(cat_test).to(device)

In [20]:
cat_train

tensor([[2, 2, 0,  ..., 0, 2, 0],
        [1, 1, 0,  ..., 0, 2, 1],
        [0, 0, 1,  ..., 1, 2, 0],
        ...,
        [1, 2, 0,  ..., 1, 2, 1],
        [2, 1, 1,  ..., 1, 2, 1],
        [2, 2, 0,  ..., 1, 2, 0]], device='mps:0')

In [21]:
cont_train = np.stack([train_df[col].values for col in num_cols],axis=1)
cont_train = torch.Tensor(cont_train).to(device)
cont_test = np.stack([test_df[col].values for col in num_cols],axis=1)
cont_test = torch.Tensor(cont_test).to(device)

In [22]:
cont_test

tensor([[13., 63.,  8., 52.,  2.,  4.],
        [ 8., 88.,  8., 92.,  1.,  4.],
        [23., 81.,  7., 85.,  0.,  4.],
        ...,
        [22., 67., 10., 55.,  1.,  2.],
        [24., 78.,  8., 87.,  2.,  3.],
        [17., 85.,  7., 94.,  4.,  3.]], device='mps:0')

In [23]:
y_train = torch.Tensor(train_df[target_col].values).to(device)
y_test = torch.Tensor(test_df[target_col].values).to(device)

In [24]:
y_train

tensor([[65.],
        [82.],
        [69.],
        ...,
        [66.],
        [88.],
        [69.]], device='mps:0')

In [25]:
cat_szs = [len(df[col].cat.categories) for col in cat_cols]
emb_szs = [(size,min(50,(size+1)//2)) for size in cat_szs]

In [54]:
class Model(nn.Module):
    def __init__(self,emb_szs,n_cont,out_sz,layers,p):
        super().__init__()
        #dealing with cat
        self.embeds = nn.ModuleList([nn.Embedding(ni,nf) for ni,nf in emb_szs])
        self.em_drop = nn.Dropout(p)
        n_emb = sum([nf for ni,nf in emb_szs])
        #for cont
        self.bn_cont = nn.BatchNorm1d(n_cont)
        #sizes 
        n_in = n_emb + n_cont
        
        #creating layers
        layerlist = []
        for i in layers:
            #one layer consists of Linear() --> ReLU() --> BatchNorm() --> Dropout()
            layerlist.append(nn.Linear(n_in,i))
            layerlist.append(nn.ReLU(inplace=True))
            layerlist.append(nn.BatchNorm1d(i))
            layerlist.append(nn.Dropout(p))
            n_in = i
        layerlist.append(nn.Linear(layers[-1],out_sz))
        #create layers attribute
        self.layers = nn.Sequential(*layerlist)
    #construction of NN is complete
    def forward(self,x_cat,x_cont):
        #process categorical with embeddings
        embeddings = []
        for i,e in enumerate(self.embeds):
            embeddings.append(e(x_cat[:,i]))
        x = torch.cat(embeddings,1)
        x = self.em_drop(x)
        #process cont variables
        x_cont = self.bn_cont(x_cont)
        x = torch.cat([x,x_cont],1)
        #both have been processed, pass data through layers
        x = self.layers(x)
        return x
        

In [116]:
model = Model(emb_szs,cont_train.shape[1],1,[256,64,32,16],p=0.2).to(device)

In [117]:
num_epochs = 6000
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)
criterion = nn.MSELoss()

In [118]:
import time
start_time = time.time()

for epoch in range(num_epochs):
    y_pred = model.forward(cat_train,cont_train)
    loss = torch.sqrt(criterion(y_pred,y_train))#RMSE
    if (epoch+1)%500 == 0:
        print(f"epoch : {epoch+1}/{num_epochs} loss : {loss.item()}")
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
duration = time.time() - start_time
print(f'Training duration : {duration/60} mins')

epoch : 500/6000 loss : 8.261551856994629
epoch : 1000/6000 loss : 7.929591178894043
epoch : 1500/6000 loss : 7.734521389007568
epoch : 2000/6000 loss : 7.529820919036865
epoch : 2500/6000 loss : 6.988405227661133
epoch : 3000/6000 loss : 6.460260391235352
epoch : 3500/6000 loss : 5.866971492767334
epoch : 4000/6000 loss : 5.489711284637451
epoch : 4500/6000 loss : 4.844503402709961
epoch : 5000/6000 loss : 4.283199310302734
epoch : 5500/6000 loss : 3.7090206146240234
epoch : 6000/6000 loss : 3.1000514030456543
Training duration : 0.7233144481976826 mins


In [126]:
with torch.no_grad():
    y_eval = model(cat_test,cont_test)
print(torch.sqrt(criterion(y_eval,y_test)).item())

3.6074280738830566
